In [1]:
# Import
import time
import os
import pickle
import csv
import pandas as pd
import torch
from torch.utils.data import Subset
from pprint import pprint
from dataclasses import dataclass

from helper.models import load_model_with_weights

# Main Analysis Function
from typing import Tuple
from helper.global_variables import DEVICE, SUS_YAML_PATH, EXPERIMENT_YAML_PATH, TRAIN_YAML_PATH
from helper.config import SuspiciousnessConfig, ExperimentConfig, TrainConfig
from helper.models import get_layer, get_model
from helper.datasets import get_dataset_loader
from helper.suspiciousness import get_ochiai, get_tarantula
from helper.suspiciousness import collect_activations, compute_hit_spectrum, log_diagnostics
from helper.general import ensure_directory_exists, empty_gpu_cache
from helper.suspiciousness import HitSpectrum


ModuleNotFoundError: No module named 'global_variables'

In [2]:
# Load Config
train_config = TrainConfig(TRAIN_YAML_PATH)
sus_config = SuspiciousnessConfig(SUS_YAML_PATH)
exp_config = ExperimentConfig(EXPERIMENT_YAML_PATH, train_config)

print(exp_config.exp_runs)
# Parse Layer Config
pprint(f"Model layers of interest: {exp_config.get_relevant_model_layer_configs()}")

[<helper.config.ExperimentRun object at 0x7fa49e521850>, <helper.config.ExperimentRun object at 0x7fa328876f90>, <helper.config.ExperimentRun object at 0x7fa49e5216d0>, <helper.config.ExperimentRun object at 0x7fa3287b26f0>, <helper.config.ExperimentRun object at 0x7fa3287b2c00>, <helper.config.ExperimentRun object at 0x7fa3287b3350>, <helper.config.ExperimentRun object at 0x7fa3287b3290>, <helper.config.ExperimentRun object at 0x7fa3287b3bf0>, <helper.config.ExperimentRun object at 0x7fa3287b2e40>, <helper.config.ExperimentRun object at 0x7fa3287b3a40>, <helper.config.ExperimentRun object at 0x7fa3287b3440>, <helper.config.ExperimentRun object at 0x7fa3287b30e0>, <helper.config.ExperimentRun object at 0x7fa3287b3740>, <helper.config.ExperimentRun object at 0x7fa3287b3620>, <helper.config.ExperimentRun object at 0x7fa3287b3260>, <helper.config.ExperimentRun object at 0x7fa3287b35f0>, <helper.config.ExperimentRun object at 0x7fa3287b3650>, <helper.config.ExperimentRun object at 0x7fa328

In [3]:
# TODO: Refactor this

# Helper Functions
def get_test_loader_wrapper(model_name: str) -> torch.utils.data.DataLoader:
    # model match
    if model_name.startswith("mnist-"):
        _, test_loader = get_dataset_loader("mnist")
    elif model_name.startswith("fmnist-"):
        _, test_loader = get_dataset_loader("fmnist")
    elif model_name.startswith("cifar10-"):
        _, test_loader = get_dataset_loader("cifar10")
    else:
        raise Exception(f"Unknown dataset: {model_name}")
    return test_loader

In [ ]:
# Hit Spectrum Helper

def unsqueeze_tensors(activations: torch.Tensor,
                      targets: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    activations_copy = activations.clone()
    targets_copy = targets.clone()
    
    # Handle shapes based on layer type
    if len(activations.shape) == 4:  # Conv layer
        # For conv layer, we want to analyze each feature map position
        # Reshape activations to (batch_size, channels * height * width)
        b, c, h, w = activations_copy.shape
        activations_copy = activations_copy.view(b, c * h * w)
        targets_copy = targets_copy.unsqueeze(1).expand(-1, c * h * w)
    else:  # Dense layer
        targets_copy = targets_copy.unsqueeze(1).expand(-1, activations_copy.shape[1])
    
    return activations_copy, targets_copy

def hs_analysis(model, layer, test_loader, target_class=0):
    """
    Analyze a specific layer of a model and compute hit spectrum.
    """
    # Collect activations
    activations, target_preds = collect_activations(model, layer, test_loader, target_class)

    # Unsqueeze tensors
    activations, target_preds = unsqueeze_tensors(activations, target_preds)
    
    # Compute hit spectrum
    hs = compute_hit_spectrum(activations, target_preds, sus_config.activation_threshold)
    
    if sus_config.verbose:
        log_diagnostics(activations, 
                        target_preds, 
                        activations > sus_config.activation_threshold, 
                        hs)
    
    return hs

In [5]:
# Result Helpers

@dataclass 
class SuspiciousnessResult:
    ochiai: torch.Tensor
    tarantula: torch.Tensor
    duration: float

def save_sus_values(sus_path: str, sus_results: SuspiciousnessResult):
    dirname = os.path.dirname(sus_path)
    ensure_directory_exists(dirname)
    with open(sus_path, 'wb+') as file:
            # process your file
            sus = (sus_results.ochiai, sus_results.tarantula)
            pickle.dump(sus, file)    

In [6]:
times = []

for model_name, layer_config in exp_config.get_relevant_model_layer_configs():

    # Load model
    model = load_model_with_weights(train_config.look_up_path(model_name))
    
    # Start timer
    time_pre = time.perf_counter()
    
    # Get test dataset loader
    test_loader = get_test_loader_wrapper(model_name)
    
    # Get Sub Dataset
    test_dataset = test_loader.dataset
    
    if len(test_dataset) > sus_config.samples:
        sub_test_dataset = Subset(test_dataset, range(sus_config.samples))
    else:
        raise Exception("Desired dataset does not have enough samples.")

    # Shuffle enabled
    sub_test_loader = torch.utils.data.DataLoader(sub_test_dataset, batch_size=64, shuffle=True)

    layer_func = get_layer(model_name, layer_config)
    layer = layer_func(model)
    
    if layer is None:
        raise Exception("Unknown layer config")
    
    # Compute hit spectrum
    hs = hs_analysis(
        model=model,
        layer=layer,
        test_loader=sub_test_loader,
        target_class=sus_config.target_class
    )

    # Compute suspiciousness
    ochiai_flattened = get_ochiai(hs)
    ochiai = ochiai_flattened.reshape(hs.layer_shape)

    tarantula_flattend = get_tarantula(hs)
    tarantula = tarantula_flattend.reshape(hs.layer_shape)
    
    # End timer
    time_diff = time.perf_counter() - time_pre
    
    # Store results
    sus_result = SuspiciousnessResult(ochiai, tarantula, time_diff)
    times.append(sus_result)
    
    # Save results
    sus_path = sus_config.target_dir + f"{model_name}/" + f"layer-config-{layer_config}.pickle"
    save_sus_values(sus_path, 
                    sus_result)

    empty_gpu_cache()
    
# Stored Times
df = pd.DataFrame(times)
if df.empty:
    raise ValueError("No valid results to save")
    
df = df.sort_values(['model_name', 'layer_config'])

ensure_directory_exists("results-adv-gen")
df.to_csv(os.path.join("results-adv-gen", 'sus-comp-times.csv'), index=False)

/home/d.rashedi/repositories/sus-adv-gen/helper/models.py:285: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(model_path)


NameError: name 'DEVICE' is not defined